# Rubik's Cube SAT Solver trial

## Modélisation: 

Le problème est un modèle dynamique, comme le cube prend différents états au cours du temps, suivant les actions qui sont choisies.
Il est donc nécessaire de prendre en compte la dimension temporelle dans notre problème, comme nous ne cherchons pas à déterminer un état final (car on connait l'état final), mais plutôt à déterminer les actions à effectuer pour passer d'un état initial à un état final.

Il faut donc modéliser d'une part les états du cube  et d'autre part les actions choisies à chaques étapes.

### Modélisation du cube : 

Pour chaque étapes, on modélise le cube en numérotant ses faces de 1 à 6, et en numérotant les cases de 1 à 4 pour chaques faces. Il nous faut aussi encoder la couleur de chaque case pour cela plusieurs choix s'offrent à nous:

![Rubik's Cube faces](./images/rubikscube1.drawio.png)
![Rubik's Cube facettes](./images/rubikscube2.drawio.png)

- En utilisant 6 variables pour chaques faces, chacune encodant la couleur. Alors seulement une variable parmis les 6 pourrat être vraie, et les autres fausses (c'est une contrainte). Cela donne $6*4*6=144$ variables pour chaques étapes.

- ou en encodant la couleur sur 3 bits, ce qui donne 8 couleurs possibles, et en interdisant 2 états pour réduire le nombre à 6 couleurs possible. Cela donne $6*4*3=72$ variables pour chaques étapes.

Pour commencer nous utiliserons la première méthode à 6 couleurs pour simplifier l'implémentation, et nous pourrons essayer la deuxième méthode si le temps de calcul est trop long.

### Modélisation des actions :

Pour modéliser les action du cubes il faut en premier lieu dénombrer le nombre d'actions possibles. Il y a en tout 18 mouvements posssibles, pour chaque face on peut faire une rotation de 90° dans le sens horaire ou anti-horaire, ou une rotation de 180°, ce qui donne $6*3=18$ actions possibles.

En réalité beaucoup sont redondantes. Par exemple, si on fait une rotation de 90° dans le sens horaire de la face 1, cela revient à faire une rotation de 270° dans le sens anti-horaire de la face 1, ou une rotation de 90° dans le sens anti-horaire de la face 6. On peut réduire le nombre d'actions possible à différents niveau :

- 3 actions : rotation de 90° dans le sens horaire, pour la face du haut, du devant et de gauche (les faces 1,2,3). avec ces 3 actions on peut réaliser toutes les actions, c'est la modélisation minimale. Cependant cela ne nous permet pas de trouver le nombre d'actions minimales pour résoudre le cube, car il faudra plusieur actions répétés pour réaliser d'autres actions.

- 6 actions : pareil en rajoutant les rotations de 90° dans le sens antihoraire pour chaque face

- 9 actions : la manière optimale, en rajoutant les rotations de 180° pour les 3 faces. Avec ces 9 actions on peut réaliser toutes les actions possibles en 1 coup.

Nous utiliserons donc 9 actions pour modéliser les actions du cube, soit les rotations de 90° dans les 2 sens et les rotations de 180° pour chaque face.

Pour encoder ces 9 actions dans des variables propositionnelles, il existe plusieur manières:

- avec 9 variables propositionnelles, une pour chaque action

- avec 6 variables propositionnelle, 3 pour le choix de la face à tourner et 3 pour le choix du mouvement à réaliser (90° horaire, 90° anti-horaire, 180°)

- avec 4 variables en encodant le choix de la face à tourner sur 2 bits et le choix du mouvement sur 2 bits. ou en encodant directement le choix du mouvement sur 4 bits. Cette méthode est la plus compacte, mais elle est bien plus difficile à implémenter.

Nous utiliserons la première ou la 2eme méthode selon ce qui semble le plus simple à implémenter dans un premier temps.

### Modélisation de la résolution du cube : 

Il est important de remarquer que 

2 options s'offrent encore à nous:

- Proposition 1: on fixe le nombre d'étape, en commenceant par 1 étape. On implémente les contraintes CNF pour que le problème soit satisfait à la dernière étape. On résout le problème grâce au solveur, et si le problème est insatisfiable on recommence en incrémentant le nombre d'étape de 1, jusqu'à résolution du problème. Cette méthode nous donnera alors la solutions la plus courte possible.

- Proposition 2 : on fixe le nombre d'étape à 11, comme on sait que le cube est résolvable en 11 étapes maximum (littérature). Et on ajoute 11 variables au problème une variable i était vrai si à l'étape i le cube est résolu. Il faudra alors au moins 1 des variables i vraie pour que le problème soit satisfait. Cette modélisation ne donnera pas forcément la solution la plus courte, sauf peut être en ajoutant des contraintes supplémentaires (on ne voit pas encore lesquelles).

Pour avoir la solution optimale, nous allons nous concentrer sur le choix 1 pour l'instant.

Pour la contrainte de résolution, on force les 6 faces à avoir une unique couleur pour chacune des 6 cases. On peut aussi ajouter des contraintes pour forcer les couleurs des cases à être celles de l'état final, mais ça surcontraindrait le problème car la configuration finale est possible pour 24 état différents selons comment on tourne le cube, hors on cherche juste à résoudre le cube et pas un état final précis.


## Implémentation:


### Les variables propositionnelles:

In [15]:
# implémentation de la modélisation du probmème

import numpy as np 

t_max = 2 # (pour l'instant)


num_to_actions = ["90", "-90", "180"]
num_to_faces = ["U", "D", "F", "B", "L", "R", "M", "E", "S"]

actions_to_num = { action : i for i, action in enumerate(num_to_actions) }
faces_to_num = { face : i for i, face in enumerate(num_to_faces) }
       

def variables(n_steps):
    n_faces = 6
    n_cases = 9
    n_couleurs = 6
    R = np.arange(1, (n_steps+1)*n_faces*n_cases*n_couleurs + 1).reshape((n_steps+1), n_faces, n_cases, n_couleurs)

    # les actions (3 mouvements et 3 faces)

    
    n_mouvements = len(num_to_actions)
    n_faces_action = len(num_to_faces)
    # print("n_mouvements:", n_mouvements)
    # print("n_faces_action:", n_faces_action)
    A = np.arange(1, n_steps*n_mouvements  + 1).reshape(n_steps, n_mouvements)
    A += np.max(R)
    F = np.arange(1, n_steps*n_faces_action  + 1).reshape(n_steps, n_faces_action)
    F += np.max(A)
    
    return R, A, F
    
R, A, F = variables(t_max)

print("on s'assure que 2 variables n'ont pas le même nombre: ")
print("R :",R.min(),"->", R.max(), R.shape)
print("A :",A.min(),"->", A.max(), A.shape)
print("F :",F.min(),"->", F.max(), F.shape)
print("on a donc", F.max(), "variables propositionnelles pour notre modélisation.")



on s'assure que 2 variables n'ont pas le même nombre: 
R : 1 -> 972 (3, 6, 9, 6)
A : 973 -> 978 (2, 3)
F : 979 -> 996 (2, 9)
on a donc 996 variables propositionnelles pour notre modélisation.


### Contraintes d'unicité des mouvements:

In [16]:
# CONTRAINTES DE BASE POUR LE CUBE

# en général on utilisera 
# t pour le numéro de l'étape
# f pour la face
# i pour le numéro de la case
# c pour la couleur

def contrainte_unique_couleurs(R):
    
    t_max, n_faces, n_cases, n_couleurs = R.shape

    basic_R_constraints = []

    # chaque case doit être colorié d'au moins une couleur
    for t,f,i in np.ndindex((t_max, n_faces, n_cases)):
        basic_R_constraints.append([int(R[t,f,i,c]) for c in range(6)])

    # chaque case doit être colorié d'au plus une couleur
    for t,f,i in np.ndindex((t_max,n_faces,n_cases)):
        for k in range(n_couleurs):
            for l in range(k+1,n_couleurs):
                basic_R_constraints.append([-int(R[t,f,i,k]), -int(R[t,f,i,l])])
    
    return basic_R_constraints
            
basic_R_clauses_color = contrainte_unique_couleurs(R)
print("nombre de clauses pour l'unicité des couleurs:", len(basic_R_clauses_color))



nombre de clauses pour l'unicité des couleurs: 2592


### Contraintes d'unicité des mouvements:

In [17]:
# CONTRAINTES DE BASE POUR LES MOUVEMENTS

def contraintes_unique_mouvement(M,F):
    t_max, n_mouvements = M.shape
    t_max, n_faces_actions = F.shape

    basic_moves_constraints = []

    # il doit y avoir au moins un mouvement sur une face à chaque étape,
    for t in range(t_max):
        basic_moves_constraints.append([int(M[t,k]) for k in range(n_mouvements)])
        basic_moves_constraints.append([int(F[t,k]) for k in range(n_faces_actions)])
        
        
    # il doit y avoir au plus un mouvement sur une face à chaque étape
    for t in range(t_max):
        for k in range(n_mouvements):
            for l in range(k+1,n_mouvements):
                basic_moves_constraints.append([-int(M[t,k]), -int(M[t,l])])
        for k in range(n_faces_actions):
            for l in range(k+1,n_faces_actions):
                basic_moves_constraints.append([-int(F[t,k]), -int(F[t,l])])
    
    return basic_moves_constraints
            
basic_R_clauses_move = contraintes_unique_mouvement(A,F)
print("nombre de contraintes pour l'unicité des mouvements:", len(basic_R_clauses_move))

nombre de contraintes pour l'unicité des mouvements: 82


### Contraintes de résolution du cube:

In [18]:
# Contraite de résolution du problème

# à la dernière étape, pour chaque face, 
# les 3 premières facettes doivent être de la même couleur que la 4ème

# on peut mettre la contrainte sous forme d'égalité, mais cela nous rajoutera 2 contrainte par égalité,
# à la place ou peut simplement la mettre sous forme d'implication: 
# "pour 2 facettes A et B, pour toutes couleurs k, si A[k] alors B[k]"
# comme chaque case est déjà contrainte à n'avoir qu'une seule et même couleur, cela revient au même.
# on utilisera donc toujours l'implication au lieu de l'égalité, ce qui divise le nombre de contraintes par 2.

# on peut le prouver assez facilement:
# pour k tel que Ak = 1,  Ak => Bk , alors Bk = 1 , et ensuite pour tout f !=k , Af=0 & Bf=0,

def equal_to(a,b):
    return [[-a,b],[a,-b]]

def implies(a,b):
    return [[-a,b]]

def contraintes_de_resolution(R):
    t_max, n_faces, n_cases, n_couleurs = R.shape
    resolution_clauses = []
    for f in range(n_faces):
        for c in range(n_cases-1):
            for k in range(n_couleurs):
                resolution_clauses += implies(R[t_max-1,f,c,k], R[t_max-1,f,n_cases-1,k])
    return resolution_clauses

resolution_constraints = contraintes_de_resolution(R)
print("nombre de contraintes de résolution:", len(resolution_constraints))

nombre de contraintes de résolution: 288


### Contraintes de mouvement

Pour appliquer les contraintes de mouvements (cf de rotation), il va faloir définir chacun des mouvements pour chacunes des faces, pour nous simplifier le travail, on remodélise les 3 faces de la même façon pour n'avoir à définir les 3 mouvements différents que pour une seule face.

on renomme les facettes de la façon suivante pour chaque face (par exemple pour la face 2):

![Rubik's Cube facettes](./images/rubikscube3.drawio.png)

On rappelle la modélisation du cube : 

![Rubik's Cube faces](./images/rubikscube1.drawio.png)
![Rubik's Cube facettes](./images/rubikscube2.drawio.png)

In [19]:
# rotation horaire face 1

# on cherche à traduire des implications découlant des actions choisies


# exemple sens horaire : 
# F(t,1) \and M(t,1) => R(t,1,1,:) = R(t+1,1,2,:)
# on peut déjà le transformer en une implication comme expliqué plus haut:
# F(t,1) \and M(t,1) => ( R(t,1,1,:) => R(t+1,1,2,:) ) 
# on peut la transformer en forme normale conjonctive:
# ( -F(t,1) \or -M(t,1) \or -R(t,1,1,:) \or R(t+1,1,2,:) )


# les mouvements sont numérotés comme suis:
adjacences = {
    0 : ((4, 2), (4, 1), (4, 0), (3,2), (3, 1), (3, 0), (2, 2), (2, 1), (2, 0), (1, 2), (1, 1), (1, 0)), # Rotate Up
    1 : ((0, 6), (0, 7), (0, 8), (2, 8), (2, 5), (2, 2), (5, 2), (5, 1), (5, 0), (4, 0), (4, 3), (4, 6)), # Rotate Front
    2 : ((0, 0), (0, 3), (0, 6), (3, 8), (3, 5), (3, 2), (5, 0), (5, 3), (5, 6), (1, 0), (1, 3), (1, 6)), # Rotate Left
    3 : ((0, 2), (0, 1), (0, 0), (4, 8), (4, 5), (4, 2), (5, 6), (5, 7), (5, 8), (2, 0), (2, 3), (2, 6)), # Rotate Back
    4 : ((0, 8), (0, 5), (0, 2), (1, 8), (1, 5), (1, 2), (5, 8), (5, 5), (5, 2), (3, 0), (3, 3), (3, 6)), # Rotate Right
    5 : ((1, 6), (1, 7), (1, 8), (2, 6), (2, 7), (2, 8), (3, 6), (3, 7), (3, 8), (4, 6), (4, 7), (4, 8)), # Rotate Down
    6 : ((4, 5), (4, 4), (4, 3), (3, 5), (3, 4), (3, 3), (2, 5), (2, 4), (2, 3), (1, 5), (1, 4), (1, 3)), # Rotate Middle from Upper view
    7 : ((0, 3), (0, 4), (0, 5), (2, 7), (2, 4), (2, 1), (5, 5), (5, 4), (5, 3), (4, 1), (4, 4), (4, 7)), # Rotate Middle from Front view
    8 :  ((0, 1), (0, 4), (0, 7), (3, 7), (3, 4), (3, 1), (5, 1), (5, 4), (5, 7), (1, 1), (1, 4), (1, 7)), # Rotate Middle from Lefty view
}


increment_adjacences = {
    "90" : lambda i : (i-3) % 12,
    "-90" : lambda i : (i+3) % 12,
    "180" : lambda i : (i+6) % 12
}

increment_faces = {
    "90" : lambda i : (i-2) % 8,
    "-90" : lambda i : (i+2) % 8,
    "180" : lambda i : (i+4) % 8
}


def rotate_face_logic(how="90", action_num=0, step=1, R=None, M=None, F=None):
    
    assert action_num < 6, "action_num doit être inférieur à 6"
    assert how in ["90", "-90", "180"], "how doit être dans ['90', '-90', '180']"
    C = [] #clauses
    
    # face principale:
    ordre = [0,3,6,7,8,5,2,1]
    for i in range(8):
        f1, c1 = action_num, ordre[i]
        f2, c2 = action_num, ordre[increment_faces[how](i)]
        for k in range(6):
            C.append([-F[step,action_num], -M[step,actions_to_num[how]], -R[step,f1,c1,k], R[step+1,f2,c2,k]])
    C.append([-F[step,action_num], -M[step,actions_to_num[how]], -R[step,action_num,4,0], R[step+1,action_num,4,0]])
    
    
    # facettes adjacentes:
    ordre = adjacences[action_num]
    for i in range(len(ordre)):
        f1, c1 = ordre[i]
        f2, c2 = ordre[increment_adjacences[how](i)]
        for k in range(6):
            C.append( [ -F[step,action_num], -M[step,actions_to_num[how]], -R[step,f1,c1,k], R[step+1,f2,c2,k] ]) 
            
    # other faces:
    for face in range(6):
        if face != action_num:
            for c in range(9):
                if (face,c) not in adjacences[action_num]:
                    for k in range(6):
                        C.append([-F[step,action_num],-M[step,actions_to_num[how]], -R[step,face,c,k], R[step+1,face,c,k]])
    return C




def rotate_middle(how="90", action_num=6, step=1, R=None, M=None, F=None):
    
    assert action_num >= 6, "action_num doit être supérieur ou égal à 6"
    assert how in ["90", "-90", "180"], "how doit être dans ['90', '-90', '180']"
    
    C = [] #clauses
    
    # facettes adjacentes:
    ordre = adjacences[action_num]
    for i in range(len(ordre)):
        f1, c1 = ordre[i]
        f2, c2 = ordre[increment_faces[how](i)]
        for k in range(6):
            C.append( [ -F[step,action_num], -M[step,actions_to_num[how]], -R[step,f1,c1,k], R[step+1,f2,c2,k] ]) 
            
    # other faces:
    for face in range(6):
        for c in range(9):
            if (face,c) not in adjacences[action_num]:
                for k in range(6):
                    C.append([-F[step,action_num],-M[step,actions_to_num[how]], -R[step,face,c,k], R[step+1,face,c,k]])
                    
    return C



def contraintes_de_mouvements(R,M,F):
    t_max, n_faces, n_cases, n_couleurs = R.shape
    n_steps, n_actions = F.shape
    
    mouvement_clauses = []

    for t in range(t_max-1):
        for action_num in range(n_actions):
            for how in ["90", "-90", "180"]:
                if action_num < 6:
                    mouvement_clauses += rotate_face_logic(how=how,action_num=action_num,step=t,R=R,M=M,F=F)
                else:
                    mouvement_clauses += rotate_middle(how=how,action_num=action_num,step=t,R=R,M=M,F=F)
                                
    return mouvement_clauses

mouvement_clauses = contraintes_de_mouvements(R,A,F)
print("nombre de clauses:", len(mouvement_clauses))


nombre de clauses: 17316


### Contraintes de départ

Pour finaliser le jeu de contraintes, il suffit de forcer les couleurs à être dans un état initial donné. Nous devons alors créer un rubik's cube résolvable (ou non) pour tester notre solveur. Pour cela nous allons utiliser un cube résolu, et nous allons le mélanger aléatoirement pour obtenir un état initial.

Pour parler des contraintes plus précisement, il faudra une contrainte par cette, obligeant la couleur de la case à être celle de l'état initial. 

In [20]:
import random


COLORS = {
    0: "🟥",  # Rouge (Up)
    1: "🟩",  # Vert (Front)
    2: "🟦",  # Bleu (Left)
    3: "🟧",  # Orange (Back)
    4: "🟨",  # Jaune (Down)
    5: "⬜"   # Blanc (Right)
}


text_2_face = {
    "Up": 0,
    "Front": 1,
    "Left": 2,
    "Back": 3,
    "Right": 4,
    "Down": 5,
}

# Création du cube 2x2x2 correctement formaté (6 faces, 4)
def create_cube():
    return np.array([[i, i, i, i, i, i, i, i, i] for i in range(6)])

# Affichage amélioré du cube en format "déplié" pour un cube 3x3x3
def print_cube(cube):
    print(f"          {COLORS[int(cube[0][0])]} {COLORS[int(cube[0][1])]} {COLORS[int(cube[0][2])]}")
    print(f"          {COLORS[int(cube[0][3])]} {COLORS[int(cube[0][4])]} {COLORS[int(cube[0][5])]}")
    print(f"          {COLORS[int(cube[0][6])]} {COLORS[int(cube[0][7])]} {COLORS[int(cube[0][8])]}\n")

    for i in range(3):
        print(f"{COLORS[int(cube[2][i*3])]} {COLORS[int(cube[2][i*3+1])]} {COLORS[int(cube[2][i*3+2])]}  "
            f"{COLORS[int(cube[1][i*3])]} {COLORS[int(cube[1][i*3+1])]} {COLORS[int(cube[1][i*3+2])]}  "
            f"{COLORS[int(cube[4][i*3])]} {COLORS[int(cube[4][i*3+1])]} {COLORS[int(cube[4][i*3+2])]}  "
            f"{COLORS[int(cube[3][i*3])]} {COLORS[int(cube[3][i*3+1])]} {COLORS[int(cube[3][i*3+2])]}")
    print()

    print(f"          {COLORS[int(cube[5][0])]} {COLORS[int(cube[5][1])]} {COLORS[int(cube[5][2])]}")
    print(f"          {COLORS[int(cube[5][3])]} {COLORS[int(cube[5][4])]} {COLORS[int(cube[5][5])]}")
    print(f"          {COLORS[int(cube[5][6])]} {COLORS[int(cube[5][7])]} {COLORS[int(cube[5][8])]}\n")

def move_edge(cube, from_face, from_indices, to_face, to_indices, temp):
    """Déplace les éléments d'un bord d'une face à une autre."""
    for i in range(3):
        cube[to_face][to_indices[i]] = temp[from_face][from_indices[i]]
    
def rotate_principal_face( cube, face ):
    cube[face] = [ cube[face][6], cube[face][3], cube[face][0], cube[face][7], cube[face][4], cube[face][1], cube[face][8], cube[face][5], cube[face][2] ]

def rotate_face(cube, face):
    """
    Effectue une rotation de 90° dans le sens horaire sur la face spécifiée et met à jour les bords.
    """
    temp = cube.copy()
    
    if face == text_2_face["Front"] or face == text_2_face["Back"]:  # Face avant
        # Mise à jour des bords (Haut → Droite, Droite → Bas, Bas → Gauche, Gauche → Haut)
        line_up = [6, 7, 8] if face == text_2_face["Front"] else [0, 1, 2]
        line_right = [0, 3, 6] if face == text_2_face["Front"] else [2, 5, 8]
        line_down = [2, 1, 0] if face == text_2_face["Front"] else [8, 7, 6]
        line_left = [8, 5, 2] if face == text_2_face["Front"] else [2, 5, 8, 6, 3, 0]

        for from_face, to_face, from_line, to_line in [(text_2_face["Up"], text_2_face["Right"], line_up, line_right),
                                                    (text_2_face["Right"], text_2_face["Down"], line_right, line_down),
                                                    (text_2_face["Down"], text_2_face["Left"], line_down, line_left),
                                                    (text_2_face["Left"], text_2_face["Up"], line_left, line_up)]:
            move_edge(cube, from_face, from_line, to_face, to_line, temp)
        

    if face == text_2_face["Up"] or face == text_2_face["Down"]:  # Face Haut
        line = [0, 1, 2] if face == text_2_face["Up"] else [6, 7, 8]
        for from_face, to_face in [(text_2_face["Front"], text_2_face["Left"]), (text_2_face["Left"], text_2_face["Back"]),
                                (text_2_face["Back"], text_2_face["Right"]), (text_2_face["Right"], text_2_face["Front"])]:
            move_edge(cube, from_face, line, to_face, line, temp)
        
        
    if face == text_2_face["Left"] or face == text_2_face["Right"]: #Face gauche
        line = [0, 3, 6] if face == text_2_face["Left"] else [8, 5, 2]
        back_line = [8, 5, 2] if face == text_2_face["Left"] else [0, 3, 6]
        for from_face, to_face, from_line, to_line in [(text_2_face["Up"], text_2_face["Back"], line, line),
                                                    (text_2_face["Back"], text_2_face["Down"], line, line),
                                                    (text_2_face["Down"], text_2_face["Front"], line, back_line),
                                                    (text_2_face["Front"], text_2_face["Up"], back_line, line)]:
            move_edge(cube, from_face, from_line, to_face, to_line, temp)
        
    
    
    # Rotation autour de la face
    
    rotate_principal_face(cube, face)
    
    return cube


# Exécution
cube = create_cube()
print("Cube initial :")
print_cube(cube)


n = 0 # Nombre de rotations aléatoires
for i in range(n):
    cube = rotate_face(cube, random.randint(0,5))  # Rotation de la face FRONT
    #print(f"Après {i+1} rotations :")
    #print_cube(cube)

    

print(f"Après {n} rotations :")
print_cube(cube)



# Ajouter les contraintes à notre problème

def contraintes_initiales(R, cube):
    t_max, n_faces, n_cases, n_couleurs = R.shape
    
    contraintes = []
    
    for f in range(n_faces):
        for i in range(n_cases):
            for c in range(n_couleurs):
                if cube[f][i] == c:
                    contraintes.append([R[0,f,i,c]])
    return contraintes

initial_constraints = contraintes_initiales(R, cube)
print( "nombre de contraintes initiales:", len(initial_constraints) )



Cube initial :
          🟥 🟥 🟥
          🟥 🟥 🟥
          🟥 🟥 🟥

🟦 🟦 🟦  🟩 🟩 🟩  🟨 🟨 🟨  🟧 🟧 🟧
🟦 🟦 🟦  🟩 🟩 🟩  🟨 🟨 🟨  🟧 🟧 🟧
🟦 🟦 🟦  🟩 🟩 🟩  🟨 🟨 🟨  🟧 🟧 🟧

          ⬜ ⬜ ⬜
          ⬜ ⬜ ⬜
          ⬜ ⬜ ⬜

Après 0 rotations :
          🟥 🟥 🟥
          🟥 🟥 🟥
          🟥 🟥 🟥

🟦 🟦 🟦  🟩 🟩 🟩  🟨 🟨 🟨  🟧 🟧 🟧
🟦 🟦 🟦  🟩 🟩 🟩  🟨 🟨 🟨  🟧 🟧 🟧
🟦 🟦 🟦  🟩 🟩 🟩  🟨 🟨 🟨  🟧 🟧 🟧

          ⬜ ⬜ ⬜
          ⬜ ⬜ ⬜
          ⬜ ⬜ ⬜

nombre de contraintes initiales: 54


### Résolution

In [32]:
from tqdm import tqdm


def generate_cnf_file(n_steps, cube):
    
    R, M, F = variables(n_steps)
    clauses = []
    clauses += contrainte_unique_couleurs(R)
    clauses += contraintes_unique_mouvement(M,F)
    clauses += contraintes_de_resolution(R)
    clauses += contraintes_de_mouvements(R,M,F)
    clauses += contraintes_initiales(R, cube)
    
    # print(f"pour {n_steps} steps:")
    # print("nombre de variables:", F.max())
    # print("nombre de clauses:", len(clauses))
    
    with open("cube_"+str(n_steps)+".cnf", "w") as f:
        f.write("c Generated by rubik_cube_cnf.py\n")
        f.write(f"c for the resolution of the cune in {n_steps} steps\n")
        f.write("c\n")
        f.write(f"p cnf {F.max()} {len(clauses)}\n")
        for clause in clauses:
            for variable in clause:
                f.write(str(variable) + " ")
            f.write("0\n")
    
    # print("\n")
    
#test
# generate_cnf_file(2, cube)
    
    
# Fonction pour exécuter les commandes et vérifier la sortie
def check_sat_solution():
    for n in tqdm(range(1, 12)):  # n entre 1 et 11
        
        generate_cnf_file(n, cube)
        
        #command = f"./gophersat_linux64 cube_{n}.cnf"  # Commande à exécuter (à changer en fonction de votre système d'exploitation)
        command = f"./gophersat_linux64 cube_{n}.cnf"
        
        result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Vérification si la sortie contient "SATISFIABLE"
        output = result.stdout.decode('utf-8')
        if "UNSATISFIABLE" not in output:
            print(f"SATISFIABLE trouvé pour cube_{n}.cnf")

            # Recherche de la décomposition dans la sortie
            lines = output.splitlines()
            final_line = None
            for line in lines:
                if line.startswith("v"):
                    final_line = line
                    final_line = list(map(lambda x: 0 if int(x)<0 else 1, final_line.strip(" 0/n").replace("v ", "").split(" ")))
                    # print(final_line)
                    
            return n, final_line
        else:
            pass
            # print(f"UNSATISFIABLE pour cube_{n}.cnf")

In [33]:
import subprocess



#### Génération du cube ####

n = 0 # Nombre de rotations aléatoires
cube = create_cube()
for i in range(n):
    cube = rotate_face(cube, random.randint(0,5))  # Rotation de la face FRONT
    #print(f"Après {i+1} rotations :")
    #print_cube(cube)
print(f"Cube à résoudre :")
print_cube(cube)


# Appel de la fonction pour commencer le processus
n, line = check_sat_solution()


# Analyse des résultats
Rmax = (n+1)*6*9*6
Amax = Rmax + n*3
Fmax = Rmax + n*9 

R = np.array(line[0:Rmax]).reshape((n+1,6,9,6))
# print(R)
A = np.array(line[Rmax:Amax]).reshape( (n,3) )

F = np.array(line[Amax:]).reshape( (n,9) )


print("\nSatisfiable pour", n, "étapes")

for t in range(n):
    print("step", t,end=": ")
    for i in range(9):
        if F[t,i] != 0:
            print(num_to_faces[i],end=" ")
    for i in range(3):
        if A[t,i] != 0:
            print(num_to_actions[i])


Cube à résoudre :
          🟥 🟥 🟥
          🟥 🟥 🟥
          🟥 🟥 🟥

🟦 🟦 🟦  🟩 🟩 🟩  🟨 🟨 🟨  🟧 🟧 🟧
🟦 🟦 🟦  🟩 🟩 🟩  🟨 🟨 🟨  🟧 🟧 🟧
🟦 🟦 🟦  🟩 🟩 🟩  🟨 🟨 🟨  🟧 🟧 🟧

          ⬜ ⬜ ⬜
          ⬜ ⬜ ⬜
          ⬜ ⬜ ⬜



  9%|▉         | 1/11 [00:00<00:01,  6.40it/s]

SATISFIABLE trouvé pour cube_2.cnf

Satisfiable pour 2 étapes
step 0: U -90
step 1: U 90
